# Day 13: Shuttle Search

https://adventofcode.com/2020/day/13

## Part 1

In [473]:
def readInput13(filename):
    with open(filename) as f:
        lines = [ l.strip("\n") for l in f.readlines()]
    timestamp = int(lines[0])
    buses = []
    for e in enumerate (lines[1].split(",")):
        if e[1]=="x":
            continue
        buses.append((int(e[1]),e[0]))
    return timestamp, buses

timestamp, buses = readInput13("data/day13test0.txt")
timestamp, buses = readInput13("data/input13.txt")
timestamp, buses

(1000299,
 [(41, 0),
  (37, 35),
  (971, 41),
  (17, 58),
  (13, 59),
  (23, 64),
  (29, 70),
  (487, 72),
  (19, 91)])

In [475]:
nextdep = []

for b,d in buses:
    n = (int(timestamp/b)+1)*b
    nextdep.append(n)

leave = min(nextdep)
bus = activebuses[nextdep.index(leave)]
sol1 = bus*(leave-timestamp)

print("Solution Part 1 =",sol1)

Solution Part 1 = 156


## Part 2

Solution has to be multiple of first bus number (0 rest in division). Solution + the other dtime values must be multiple of all other bus numbers. Of course bus numbers are primes, so brute force will not work...

In [483]:
def bruteForcePart2(buses):
    n = buses[0][0]
    while(True):
        n += buses[0][0]
        match = 0
        for i in range(1,len(buses)):
            q = (n+buses[i][1]) % buses[i][0] # 0 if match condition for current bus
            if q!=0:
                break
            else:
                match += 1
        if match == len(buses)-1:
            return n
            break

In [486]:
# Brute force attempt on test, should return 1068781

timestamp, buses = readInput13("data/day13test0.txt")
n = bruteForcePart2(buses)
print("Solution test Part 2 =",n)

Solution test Part 2 = 1068781


Given the extremes of the problem, this has certainly to do with modular algebra, which I know very little about (I discever its extistence during last year AOC!). Following an hint from @zar I look up the [Chinese remainder theorem](https://en.wikipedia.org/wiki/Chinese_remainder_theorem):

> If one knows the remainders of the Euclidean division of an integer n by several integers, then one can determine uniquely the remainder of the division of n by the product of these integers, under the condition that the divisors are pairwise coprime.

Solution will hopefully follow in the coming days, once I get the time to study and understand how to use this...